In [1]:
import requests
import asyncio
from bs4 import BeautifulSoup
import json
import re
import pandas as pd
from fake_useragent import UserAgent

In [ ]:
import logging
logging.basicConfig(
    format="%(asctime)s [%(levelname).7s\t] %(message)s",
    datefmt="%d/%m %H:%M:%S",
    level=logging.INFO,
)
logger = logging.getLogger()

In [2]:
ua = UserAgent()
import socks
import socket

socks.set_default_proxy(socks.SOCKS5, "localhost", 9150)
socket.socket = socks.socksocket

In [4]:
def checkIP():
    ip = requests.get('http://checkip.dyndns.org').content
    soup = BeautifulSoup(ip, 'html.parser')
    print(soup.find('body').text)

checkIP()

Current IP Address: 176.10.99.200


In [3]:
base_url = "https://www.kinopoisk.ru/user/{user_id}/votes/list/ord/date/page/{page_num}/#list"
user_id = 26711231
page_num = 1
current_status_code = 200
user_data = []

while current_status_code != 404:
    url = base_url.format(user_id = user_id, page_num = page_num)
    response = requests.get(url, headers = {'User-Agent': ua.chrome})
    current_status_code = response.status_code
    if current_status_code != 404:
        soup = BeautifulSoup(response.text, "html.parser")
        film_list = soup.find('div', {'class': 'profileFilmsList'})
        items = film_list.find_all('div', {'class': ['item', 'item even']})
        for item in items:
            movie_link = item.find('div', {'class': 'nameRus'}).find('a').get('href')
            movie_id = re.findall('\d+', movie_link)[0]
            watch_datetime = item.find('div', {'class': 'date'}).text
            watch_datetime = pd.to_datetime(watch_datetime)
            user_rating = item.find('div', {'class': 'vote'}).text
            if user_rating:
                user_rating = int(user_rating)
        
            user_data.append({
                    'movie_id': movie_id,
                    'timestamp': watch_datetime,
                    'user_rating': user_rating,
                })
            
        page_num += 1
user_data

[{'movie_id': '995974',
  'timestamp': Timestamp('2019-09-06 16:43:00'),
  'user_rating': 8}]

In [44]:
import aiohttp
import asyncio

async def main():
    async with aiohttp.ClientSession() as session:
        async with session.get('https://www.kinopoisk.ru/user/26711231/votes/list/ord/date/page/1/#list') as response:
            print("Status:", response.status)
            html = await response.text()
    return html

loop = asyncio.get_event_loop()
x = loop.run_until_complete(main())

Status: 200


In [39]:
import aiohttp
import asyncio
import nest_asyncio
nest_asyncio.apply()

async def get(url):
    async with aiohttp.ClientSession() as session:
        async with session.get(url) as response:
            return await response.text()


loop = asyncio.get_event_loop()


list_urls_test = ['https://www.kinopoisk.ru/user/26711231/votes/list/ord/date/page/1/#list', 'https://www.kinopoisk.ru/user/26718123/votes/list/ord/date/page/1/#list']
#resp = [get(url) for url in list_urls_test]

#results = loop.run_until_complete(asyncio.gather(*coroutines))
results = loop.run_until_complete(get(list_urls_test))

TypeError: Constructor parameter should be str

In [35]:
resp

[<coroutine object get at 0x00000222D389AD48>,
 <coroutine object get at 0x00000222D389A548>]

In [7]:
from aiohttp import ClientSession
from typing import List
import asyncio
import nest_asyncio
nest_asyncio.apply()


class NoDataException(BaseException):
    pass

async def fetch(url, session):
    async with session.get(url=url) as response:
        if response.status != 200:
            logger.info(f"GET {url}")
            logger.info(f"GET {response.status} {url}")
            logger.info(response)
            raise NoDataException
        return await response.read()


async def bound_fetch(sem, url, session):
    # Getter function with semaphore.
    async with sem:
        return await fetch(url, session)


async def run(urls):
    tasks = []
    # create instance of Semaphore
    sem = asyncio.Semaphore(10)

    # Create client session that will ensure we dont open new connection
    # per each request.
    async with ClientSession() as session:
        for url in urls:
            # pass Semaphore and session to every GET request
            task = asyncio.ensure_future(bound_fetch(sem, url, session))
            tasks.append(task)

        responses = asyncio.gather(*tasks)
        await responses
        return responses


def fast_requests(urls: List[str]) -> List:
    """
    Making async requests

    :param urls: URLs to make requests
    :returns: list or responses
    """
    loop = asyncio.get_event_loop()
    future = asyncio.ensure_future(run(urls))
    reqs = loop.run_until_complete(future)
    return reqs.result()

In [10]:
x = fast_requests(['https://www.kinopoisk.ru/user/26711231/votes/list/ord/date/page/1/#list', 'https://www.kinopoisk.ru/user/26718123/votes/list/ord/date/page/1/#list'])

In [13]:
x[1]

b'\n<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\n<html xmlns="http://www.w3.org/1999/xhtml" xmlns:og="http://opengraphprotocol.org/schema/" dir="ltr" lang="ru" prefix="og: http://ogp.me/ns# video: http://ogp.me/ns/video# ya: http://webmaster.yandex.ru/vocabularies/">\n<head profile="http://gmpg.org/xfn/11">\n    <title>\xd0\x9f\xd1\x80\xd0\xbe\xd1\x84\xd0\xb8\xd0\xbb\xd1\x8c: ivpahomov12 - \xd0\x9e\xd1\x86\xd0\xb5\xd0\xbd\xd0\xba\xd0\xb8</title>\n\n    <meta http-equiv="content-type" content="text/html; charset=utf-8" />\n    <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1" />\n    <meta property="fb:app_id" content="121953784483000"/>\n    <meta name="msapplication-TileColor" content="#FF6600">\n    <meta name="msapplication-TileImage" content="https://st8.kp.yandex.net/public/img/favicons/favicon-144.png">\n    <meta name="application-name" content="\xd0\x9a\xd0\xb8\xd0\xbd\xd0\xbe\xd0\xbf\xd0\